In [2]:
import snap
import numpy as np
import pandas as pd
import os
from graphviz import Graph
from graphviz import Source
import matplotlib.pyplot as plt

In [5]:
with open(os.path.realpath('input/members_mapping.csv'), 'rb') as f:
    deputats_mapping = pd.read_csv(f) 

In [97]:
with open(os.path.realpath('input/summary_id_current.csv'), 'rb') as f:
    summary_id_current = pd.read_csv(f)

### Активность в ГД

In [105]:
summary_id_current

,id,position
0,99110156,Член СФ
1,99111774,Член СФ
2,99112936,Депутат ГД
3,99111987,Депутат ГД
4,99112712,Депутат ГД
5,99109653,Депутат ГД
6,99100784,Депутат ГД
7,99100010,Член СФ
8,99111778,Депутат ГД
9,99111779,Депутат ГД


In [75]:
from corelib.data_dump import dumper

In [86]:
# Dump open (prepared dataset)
dep_info = dumper(path='dep_info', method='shelve', task='o')

In [87]:
list(dep_info)

[{'id': '99111774',
  'family': 'Абрамов',
  'name': 'Иван',
  'patronymic': 'Николаевич',
  'birthdate': '1978-06-16',
  'credentialsStart': '2016-09-18',
  'credentialsEnd': '2018-06-13',
  'factionId': '72100005',
  'factionName': 'Фракция Политической партии ЛДПР - Либерально-демократической партии России',
  'factionRole': ' 00023 Член фракции',
  'partyNameInstr': None,
  'isActual': 'false',
  'homePage': None,
  'factionRegion': None,
  'nameGenitive': 'Абрамова Ивана Николаевича',
  'lawcount': 63,
  'regions': [],
  'familyAndInitials': 'Абрамов И.Н.',
  'speachCount': 136,
  'voteLink': 'http://vote.duma.gov.ru/?convocation=AAAAAAA7&deputy=99111774&sort=date_desc',
  'transcriptLink': 'http://www.cir.ru/duma/servlet/is4.wwwmain?FormName=ProcessQuery&Action=RunQuery&PDCList=*&QueryString=%2FGD_%C4%C5%CF%D3%D2%C0%D2%3D%22%C0%C1%D0%C0%CC%CE%C2+%C8.%CD.%22',
  'educations': [],
  'degrees': [],
  'ranks': [],
  'activity': []},
 {'id': '99112936',
  'family': 'Авдеев',
  'name':

In [88]:
activity_with_id = list(dep_info).copy()

In [89]:
for i in activity_with_id:
    is_activity = i.get('activity')
    if not is_activity:
        activity_with_id.remove(i)

In [90]:
activity_with_id = pd.io.json.json_normalize(activity_with_id, record_path='activity', meta='id',
                                             record_prefix='activity.', errors="ignore")

In [92]:
activity_with_id.drop(['activity.subdivisionId'], axis='columns', inplace=True)

In [93]:
activity_with_id

,activity.name,activity.subdivisionNameGenitive,id
0,Член комитета,Комитета ГД по контролю и Регламенту,99112936
1,Член комитета,Комитета ГД по контролю и Регламенту,99112936
2,Член комитета,Комитета ГД по контролю и Регламенту,99112936
3,Заместитель председателя комитета,Комитета ГД по финансовому рынку,99111987
4,Член комитета,Комитета ГД по аграрным вопросам,99112712
5,Член комитета,Комитета ГД по безопасности и противодействию ...,99109653
6,Председатель комитета,Комитета ГД по финансовому рынку,99100784
7,Член комитета,Комитета ГД по международным делам,99111778
8,Член комитета,Комитета ГД по транспорту и строительству,99111779
9,Член комитета,Комитета ГД по контролю и Регламенту,99112719


In [100]:
pandas_profiling.ProfileReport(activity_with_id)

## Граф, в котором подчиненные указывают на начальника

In [102]:
list(activity_with_id['activity.name'].unique())

['Член комитета',
 'Заместитель председателя комитета',
 'Председатель комитета',
 'Заместитель председателя комиссии',
 'Первый заместитель председателя комитета',
 'Член комиссии',
 'Председатель комиссии',
 'Председатель ГД',
 'Заместитель Председателя ГД',
 'Первый заместитель Председателя ГД',
 'Первый заместитель председателя комиссии',
 'Сопредседатель комиссии']

In [103]:
# new directed graph
order_graph = snap.TNGraph.New()

In [106]:
for i in summary_id_current['id']:
    order_graph.AddNode(i)

In [107]:
len(list(order_graph.Nodes()))

565

In [176]:
def activity_name_filter(param, data=activity_with_id, column='activity.name'):
    table = activity_with_id.mask(activity_with_id['activity.name'] != param)
    table.dropna(how='all', inplace=True)
    table['id'] = table['id'].astype(np.int64)
    return table

In [177]:
dict_of_role = {}
for i in list(activity_with_id['activity.name'].unique()):
    dict_of_role[i] = activity_name_filter(param=i)

In [178]:
dict_of_role.keys()

dict_keys(['Член комитета', 'Заместитель председателя комитета', 'Председатель комитета', 'Заместитель председателя комиссии', 'Первый заместитель председателя комитета', 'Член комиссии', 'Председатель комиссии', 'Председатель ГД', 'Заместитель Председателя ГД', 'Первый заместитель Председателя ГД', 'Первый заместитель председателя комиссии', 'Сопредседатель комиссии'])

#### Первые зампреды ГД

In [179]:
pred = list(dict_of_role['Председатель ГД']['id'])[0]

In [181]:
first_zampred = list(dict_of_role['Первый заместитель Председателя ГД']['id'])

In [184]:
for i in first_zampred:
    order_graph.AddEdge(i, pred)
    print('Edges added')

Edges added
Edges added


#### Зампреды ГД

In [186]:
zampred = list(dict_of_role['Заместитель Председателя ГД']['id'])

In [188]:
for i in zampred:
    order_graph.AddEdge(i, pred)
    for j in first_zampred:
        order_graph.AddEdge(i, j)
    print('Edges added')

Edges added
Edges added
Edges added
Edges added
Edges added
Edges added


#### Председатели комитетов

In [189]:
pred_com = list(dict_of_role['Председатель комитета']['id'])

In [191]:
for i in pred_com:
    order_graph.AddEdge(i, pred)
    print('Edges added')

Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added


#### Председатели комиссий

In [192]:
pred_commis = list(dict_of_role['Председатель комиссии']['id'])

In [194]:
for i in pred_commis:
    order_graph.AddEdge(i, pred)
    print('Edges added')

Edges added
Edges added
Edges added
Edges added


#### Первые зампреды комитетов

In [202]:
first_zam_pred_com = dict_of_role['Заместитель председателя комитета']

In [217]:
pred_com_full = dict_of_role['Председатель комитета'].copy()

In [236]:
list_of_pred_with_id = []
for i in range(len(pred_com_full.index)):
    list_of_pred_with_id.append((pred_com_full.iloc[i]['activity.subdivisionNameGenitive'], 
                                 pred_com_full.iloc[i]['id']))

In [238]:
list_of_zam_pred_with_id = []
for i in range(len(first_zam_pred_com.index)):
    list_of_zam_pred_with_id.append((first_zam_pred_com.iloc[i]['activity.subdivisionNameGenitive'], 
                                     first_zam_pred_com.iloc[i]['id']))

In [246]:
for i, j in list_of_zam_pred_with_id:
    for k, v in list_of_pred_with_id:
        if i == k:
            order_graph.AddEdge(int(j), int(v))
            print('Edges added')

Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edges added
Edge

#### Зампреды комитетов

#### Рядовые члены комитетов

#### Сопредседатели комиссий

#### Первые зампреды комиссий

#### Зампреды комиссий

#### Рядовые члены комиссий

## Строим граф

In [247]:
snap.SaveEdgeList(order_graph, "output/order_graph.txt", "order_graph")

In [248]:
snap.DrawGViz(order_graph, snap.gvlNeato, "images/order_graph.png", "order_graph", True)